In [18]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms

from utils import *
from data_setup import *

In [19]:
device = device_check()

Using mps device


In [20]:
device = "cpu"

train_dir = "./data/pizza_steak_sushi/train"
test_dir = "./data/pizza_steak_sushi/test"

manual_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
        ])

train_dataloader, test_dataloader, classes, class_to_idx = create_dataloaders(train_dir,
                                                                              test_dir,
                                                                              manual_transform,
                                                                              32, num_workers = 0)
train_dataloader, test_dataloader, classes, class_to_idx

(<torch.utils.data.dataloader.DataLoader at 0x158fd5360>,
 ['pizza', 'steak', 'sushi'],
 {'pizza': 0, 'steak': 1, 'sushi': 2})

In [21]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_trans = weights.transforms()

train_dataloader, test_dataloader, classes, class_to_idx = create_dataloaders(train_dir,
                                                                              test_dir,
                                                                              auto_trans,
                                                                              32,
                                                                              num_workers = 0)

train_dataloader, test_dataloader, classes, class_to_idx

(<torch.utils.data.dataloader.DataLoader at 0x12f7fdf60>,
 ['pizza', 'steak', 'sushi'],
 {'pizza': 0, 'steak': 1, 'sushi': 2})

In [22]:
model = torchvision.models.efficientnet_b0(weights = weights).to(device)

for param in model.features.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(nn.Dropout(p = 0.2, inplace = True),
                                 nn.Linear(1280, 3)).to(device)

summary(model, (32, 3, 224, 224), verbose = 0,
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 15,
        row_settings = ["var_names"])


Layer (type (var_name))                                      Input Shape     Output Shape    Param #         Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224] [32, 3]         --              Partial
├─Sequential (features)                                      [32, 3, 224, 224] [32, 1280, 7, 7] --              False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224] [32, 32, 112, 112] --              False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224] [32, 32, 112, 112] (864)           False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112] [32, 32, 112, 112] (64)            False
│    │    └─SiLU (2)                                         [32, 32, 112, 112] [32, 32, 112, 112] --              --
│    └─Sequential (1)                                        [32, 32, 112, 112] [32, 16, 112, 112] --              False
│    │    └─MBConv (0)                      

In [23]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)



In [24]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()



In [31]:
def train(model: nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: nn.Module,
          epochs: int,
          device: torch.device
          ):
    results = {"train_loss": [], "train_acc":[], "test_loss":[], "test_acc":[]}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, None, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        writer.add_scalar(tag = "Train Loss", scalar_value = train_loss, global_step = epoch)
        writer.add_scalar(tag = "Test Loss", scalar_value = test_loss, global_step = epoch)
        writer.add_scalar(tag = "Train Acc", scalar_value = train_acc, global_step = epoch)
        writer.add_scalar(tag = "Test Acc", scalar_value = test_acc, global_step = epoch)

        writer.add_graph(model = model, input_to_model = torch.randn(32, 3, 224, 224).to(device))


    writer.close()
    return results


In [32]:
results = train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs = 3, device
= device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6076 | train_acc: 0.8008 | test_loss: 0.5617 | test_acc: 0.8456
Epoch: 2 | train_loss: 0.5412 | train_acc: 0.8086 | test_loss: 0.5609 | test_acc: 0.8759
Epoch: 3 | train_loss: 0.5370 | train_acc: 0.8359 | test_loss: 0.5548 | test_acc: 0.8561


In [13]:
from utils import train_step, test_step


def new_train(model: nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
               }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model = model,
                                           dataloader = train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer = optimizer,
                                           scheduler = None,
                                           device = device)
        test_loss, test_acc = test_step(model = model,
                                        dataloader = test_dataloader,
                                        loss_fn = loss_fn,
                                        device = device)

        # Print out what's happening
        print(
                f"Epoch: {epoch + 1} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_acc: {train_acc:.4f} | "
                f"test_loss: {test_loss:.4f} | "
                f"test_acc: {test_acc:.4f}"
                )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag = "Loss",
                           tag_scalar_dict = {"train_loss": train_loss,
                                              "test_loss": test_loss},
                           global_step = epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag = "Accuracy",
                           tag_scalar_dict = {"train_acc": train_acc,
                                              "test_acc": test_acc},
                           global_step = epoch)

        # Track the PyTorch model architecture
        writer.add_graph(model = model,
                         # Pass in an example input
                         input_to_model = torch.randn(32, 3, 224, 224).to(device))

    # Close the writer
    writer.close()

    ### End new ###

    # Return the filled results at the end of the epochs
    return results


In [14]:
results = new_train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/lipeiran/anaconda3/envs/TorchProject/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/lipeiran/anaconda3/envs/TorchProject/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/lipeiran/anaconda3/envs/TorchProject/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <7422C53A-9F13-3592-9338-E2EA7D46A90F> /Users/lipeiran/anaconda3/envs/TorchProject/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
/Users/lipeiran/anaconda3/envs/TorchProject/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/lipeiran/anaconda3/env

RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same